<a href="https://colab.research.google.com/github/seoharuss/AI_study/blob/main/nnModule%EC%84%A0%ED%98%95%ED%9A%8C%EA%B7%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## nn.Module로 구현하는 선형 회귀

파이토치에서 이미 구현되어져 제공되고 있는 함수들을 불러와 더 쉽게 선형 회귀 모델을 구현해보자

예를 들어 파이토치에서는 선형 회귀 모델이 `nn.Linear()`라는 함수로, MSE는 `nn.functional.mse_loss()`라는 함수로 구현되어져 있다.

In [ ]:
# 두 함수의 사용 예제
# import torch.nn as nn
# model = nn.Linear(input_dim, output_dim)

In [ ]:
# import torch.nn.functional as F
# cost = F.mse_loss(predicition, y_train)

## 단순 선형 회귀 구현하기

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

이제 데이터를 선언한다.

아래 데이터는 y=2x를 가정된 상태에서 만들어진 데이터로 이미 정답이 w=2, b=0임을 알고 있는 상태다.

모델이 이 두 w와 b값을 제대로 찾아내도록 하는 것이 목표다.

In [ ]:
# 데이터
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[2], [4], [6]])

데이터를 정의하였으니 선형 회귀 모델을 구현할 차례다.

`nn.Linear()`는 입력의 차원, 출력의 차원을 인수로 받는다.

In [ ]:
# 모델을 선언 및 초기화. 단순 선형 회귀이므로 inpud_dim=1, output_dim=1
model = nn.Linear(1,1)

torch.nn.Linear 인자로 1,1을 사용했다.

하나의 입력 x에 대해서 하나의 출력 y를 가지므로, 입력 차원과 출력 차원 모두 1을 인수로 사용

model에는 가중치 w와 편향 b가 저장되어 있다.

이 값은 `model.parameters()`라는 함수를 사용하여 볼러올 수 있다.

In [ ]:
print(list(model.parameters()))

[Parameter containing:
tensor([[-0.5625]], requires_grad=True), Parameter containing:
tensor([0.5855], requires_grad=True)]


2개의 값이 출력되는데 첫번째 값이 w고, 두번째 값이 b에 해당된다.

두 값 모두 현재는 랜덤 초기화가 되어져 있다.

그리고 두 값 모두 학습의 대상이므로 requires_grad=True가 되어져 있는 것을 볼 수 있다.

이제 옵티마이저를 정의한다.

`model.parameters()`를 사용하여 w와 b를 전달한다.

학습률(learning rate)는 0.01로 정한다.

In [ ]:
# optimizer 설정. 경사 하강법 SGD를 사용하고 learning rate를 의미하는 lr은 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [ ]:
# 전체 훈련 데이터에 대해 경사 하강법을 2,000회 반복
nb_epochs = 2000
for epoch in range(nb_epochs + 1):
  # H(x) 계산
  prediction = model(x_train)

  # cost 계산
  # 파이토치에서 제공하는 MSE 함수
  cost = F.mse_loss(prediction, y_train)

  # cost로 H(x) 개선하는 부분
  # gradient를 0으로 초기화
  optimizer.zero_grad()
  # 비용 함수를 미분하여 gradient 계산
  cost.backward() # backward 연산
  # w와 b를 업데이트
  optimizer.step()

  if epoch % 100 == 0:
    print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))

Epoch    0/2000 Cost: 24.985708
Epoch  100/2000 Cost: 0.182560
Epoch  200/2000 Cost: 0.112811
Epoch  300/2000 Cost: 0.069710
Epoch  400/2000 Cost: 0.043077
Epoch  500/2000 Cost: 0.026619
Epoch  600/2000 Cost: 0.016449
Epoch  700/2000 Cost: 0.010164
Epoch  800/2000 Cost: 0.006281
Epoch  900/2000 Cost: 0.003881
Epoch 1000/2000 Cost: 0.002398
Epoch 1100/2000 Cost: 0.001482
Epoch 1200/2000 Cost: 0.000916
Epoch 1300/2000 Cost: 0.000566
Epoch 1400/2000 Cost: 0.000350
Epoch 1500/2000 Cost: 0.000216
Epoch 1600/2000 Cost: 0.000134
Epoch 1700/2000 Cost: 0.000083
Epoch 1800/2000 Cost: 0.000051
Epoch 1900/2000 Cost: 0.000032
Epoch 2000/2000 Cost: 0.000019


학습이 완료되었다.

Cost의 값이 매우 작다.

W와 b의 값도 최적화가 되었는지 확인해보자.
- x에 임의의 값 4를 넣어 모델이 예측하는 y의 값을 확인해보자

In [ ]:
# 임의의 입력 4를 선언
new_var = torch.FloatTensor([[4.0]])

# 입력한 값 4에 대해서 예측값 y를 리턴받아서 pred_y에 저장
pred_y = model(new_var) # forward 연산
# y = 2x 이므로 입력이 4라면 y가 8에 가까운 값이 나와야 제대로 학습이 된 것
print("훈련 후 입력이 4일 때의 예측값 :", pred_y)

훈련 후 입력이 4일 때의 예측값 : tensor([[7.9912]], grad_fn=<AddmmBackward0>)


이 문제의 정답은 y = 2x가 정답이므로 y값이 8에 가까우면 w와 b의 값이 어느정도 최적화가 된 것으로 볼 수 있다.
- 실제로 예측된 값은 7.xx로 8에 매우 가깝다.

In [ ]:
# 학습 후의 w와 b의 값을 출력
print(list(model.parameters()))

[Parameter containing:
tensor([[1.9949]], requires_grad=True), Parameter containing:
tensor([0.0116], requires_grad=True)]


w의 값이 2에 가깝고, b의 값이 0에 가까운 것을 볼 수 있다.

- H(x) 식에 입력 x로부터 예측된 y를 얻는 것을 forward 연산이라고 한다.

- 학습 전, prediction = model(x_train)은 x_train으로 부터 예측값을 리턴하므로 forward 연산이다.

- 학습 후, pred_y = model(new_var)는 임의의 값 new_var로부터 예측값을 리턴하므로 forward 연산이다.

- 학습 과정에서 비용 함수를 미분하여 기울기를 구하는 것을 backward 연산이라고 한다.

- cost.backward()는 비용 함수로부터 기울기를 구하라는 의미이며 backward 연산이다.

## 다중 선형 회귀 구현


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
# 데이터 선언
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

In [ ]:
# 모델을 선언 및 초기화. 다중 선형 회귀이므로 input_dim=3, output_dim=1
model = nn.Linear(3,1)

`torch.nn.Linear()`의 인자로 3, 1을 사용했다.

3개의 입력 x에 대해서 하나의 출력 y를 가지므로 입력차원은 3, 출력차원은 1을 인수로 사용했다.

model에는 3개의 가중치 w와 편향 b가 저장되어 있다.

이 값은 model.parameters()라는 함수를 사용하여 불러올 수 있다.

In [ ]:
print(list(model.parameters()))

[Parameter containing:
tensor([[-0.1841, -0.1323, -0.1447]], requires_grad=True), Parameter containing:
tensor([-0.0263], requires_grad=True)]


첫번째 출력되는 것이 3개의 w고, 두번째 출력되는 것이 b에 해당 된다.

두 값 모두 현재는 랜덤 초기화가 되어있다.

그리고 두 출력 결과 모두 학습 대상이므로 requires_grad=True가 되어져 있다.

이제 옵티마이저를 정의한다.

`model.parameters()`를 사용하여 3개의 w와 b를 전달한다.

학습률(learning rate)는 0.00001로 정한다.
- 파이썬 코드로는 1e-5로도 표기
- 0.01로 하지 않는 이유는 기울기가 발산하기 때문

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5)

이하 코드는 단순 선형 회귀를 구현했을 때와 동일하다.

In [ ]:
nb_epochs = 2000
for epoch in range(nb_epochs + 1):
  prediction = model(x_train)
  # model(x_train)은 model.forward(x_train)와 동일
  cost = F.mse_loss(prediction, y_train)

  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch % 100 == 0:
    print('Epoch {:4d}/{} Cost: {:.6f}'.format(
          epoch, nb_epochs, cost.item()
      ))

Epoch    0/2000 Cost: 44832.523438
Epoch  100/2000 Cost: 1.865190
Epoch  200/2000 Cost: 1.784216
Epoch  300/2000 Cost: 1.707488
Epoch  400/2000 Cost: 1.634775
Epoch  500/2000 Cost: 1.565848
Epoch  600/2000 Cost: 1.500517
Epoch  700/2000 Cost: 1.438588
Epoch  800/2000 Cost: 1.379892
Epoch  900/2000 Cost: 1.324255
Epoch 1000/2000 Cost: 1.271495
Epoch 1100/2000 Cost: 1.221478
Epoch 1200/2000 Cost: 1.174083
Epoch 1300/2000 Cost: 1.129134
Epoch 1400/2000 Cost: 1.086514
Epoch 1500/2000 Cost: 1.046105
Epoch 1600/2000 Cost: 1.007792
Epoch 1700/2000 Cost: 0.971460
Epoch 1800/2000 Cost: 0.937017
Epoch 1900/2000 Cost: 0.904327
Epoch 2000/2000 Cost: 0.873338


3개의 w와 b의 값도 최적화가 되었는지 확인

In [ ]:
# 임의의 입력 선언
new_var = torch.FloatTensor([[73, 80, 75]])
# 예측값 y를 리턴 받아서 pred_y에 저장
pred_y = model(new_var)
print("훈련 후 입력이 73, 80, 75일 때의 예측값 :", pred_y)

훈련 후 입력이 73, 80, 75일 때의 예측값 : tensor([[152.2239]], grad_fn=<AddmmBackward0>)


In [ ]:
print(list(model.parameters()))

[Parameter containing:
tensor([[0.7354, 0.5816, 0.6937]], requires_grad=True), Parameter containing:
tensor([-0.0138], requires_grad=True)]
